In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from dotenv import load_dotenv
from intelligence_layer.evaluation.runner import WandbRunner
import wandb

from intelligence_layer.connectors import LimitedConcurrencyClient
from intelligence_layer.evaluation import (
    Evaluator,
    InMemoryEvaluationRepository,
    InMemoryRunRepository,
    InMemoryDatasetRepository,
    InMemoryAggregationRepository,
    Runner,
    Aggregator,
)
from intelligence_layer.evaluation.data_storage.run_repository import WandbRunRepository
from intelligence_layer.use_cases import (
    PromptBasedClassify,
    SingleLabelClassifyEvaluationLogic,
    SingleLabelClassifyAggregationLogic,
)

load_dotenv()
wandb.login(key=os.getenv("WANDB_API_KEY"))

task = PromptBasedClassify()
dataset_repository = InMemoryDatasetRepository()
run_repository = WandbRunRepository()
wandb_project_name = "test"


from intelligence_layer.core import Chunk, NoOpTracer
from intelligence_layer.use_cases import ClassifyInput
from intelligence_layer.evaluation import Example

classify_input = ClassifyInput(
    chunk=Chunk("This is good"),
    labels=frozenset({"positive", "negative"}),
)

single_example_dataset = dataset_repository.create_dataset(
    examples=[Example(input=classify_input, expected_output="positive")]
)


runner = WandbRunner(
    task,
    dataset_repository,
    run_repository,
    "Run-test",
    wandb_project_name=wandb_project_name,
)
runner.run_dataset(single_example_dataset, NoOpTracer())